In [52]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
import statsmodels.api as sm
from colorama import Fore, Style
from scipy.optimize import fsolve, curve_fit
import yfinance as yf
import akshare as ak

In [53]:
etf_code = "sh513180"  # ETF代码
start_date = "20220101"  # 开始日期
end_date = "20221231"  # 结束日期

# 调用 AKShare 的 fund_etf_hist_sina 接口获取数据
etf_hist = ak.fund_etf_hist_sina(symbol=etf_code)

etf_hist.tail()

,date,open,high,low,close,volume
933,2025-03-31,0.750,0.752,0.734,0.741,7351263171
934,2025-04-01,0.748,0.758,0.744,0.749,8068711000
935,2025-04-02,0.744,0.758,0.739,0.748,7144843500
936,2025-04-03,0.733,0.744,0.727,0.733,8748133300
937,2025-04-07,0.675,0.680,0.660,0.660,7324210938


In [54]:
a_spot = ak.stock_zh_a_spot()

# 筛选出恒生科技指数ETF（sh513180）的数据
etf_data = a_spot[a_spot["代码"] == "513180"]